<a href="https://colab.research.google.com/github/CidClayQuirino/api_als/blob/main/api_als.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

import os
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
from typing import Optional, Dict, Any, List

# ============================================
# CONFIGURAÇÕES DA API
# ============================================

LOGIN_URL = "https://integration.s360web.com/api/login"
API_BASE = "https://integration.s360web.com/api/v1"

# Use variáveis de ambiente se preferir não hardcodar credenciais
USER = os.getenv("S360_USER", "integracao.komatsu")
PASSWORD = os.getenv("S360_PASSWORD", "o8sqfl5h")

# Pasta local onde os CSV serão salvos (Ex.: Google Colab usa /content/)
PASTA_SAIDA = "/content/s360_export/"

# Período de coleta: últimos N dias (1 ano = 365)
PERIODO_DIAS = 365

# Timeout (conexão, leitura) e tentativas
TIMEOUT = (10, 60)     # (connect, read) em segundos
MAX_RETRIES = 3
RETRY_BACKOFF = 2      # base para backoff exponencial: 2, 4, 8...

# Rate limit entre chamadas (se necessário pela API)
RATE_LIMIT_SLEEP = 2   # segundos

# Configuração de CSV (mantenha ',' se preferir padrão internacional)
CSV_SEPARADOR = ","    # em pt-BR, alguns preferem ';'
CSV_ENCODING = "utf-8-sig"  # BOM para Excel (Windows)


# ============================================
# UTILITÁRIOS
# ============================================

def log_resp(r: requests.Response, prefix: str = "") -> None:
    """Loga informações úteis da resposta HTTP para diagnóstico."""
    ct = r.headers.get("Content-Type", "")
    print(f"{prefix}Status: {r.status_code} {r.reason} | Content-Type: {ct}")
    body = r.text or ""
    print(f"{prefix}Body (até 1000 chars): {body[:1000]}")
    # Cabeçalhos comumente relevantes
    importantes = ['date', 'server', 'x-request-id', 'x-correlation-id']
    print(f"{prefix}Headers: { {k: v for k, v in r.headers.items() if k.lower() in importantes} }")


def ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


# ============================================
# AUTENTICAÇÃO
# ============================================

def autenticar(session: Optional[requests.Session] = None) -> str:
    """
    Autentica e retorna o token. Faz retries para status transitórios (429/5xx)
    e detalha qualquer falha. Tenta chaves alternativas no payload se 400/401.
    """
    sess = session or requests.Session()

    payload = {"login": USER, "password": PASSWORD}
    alt_payloads = [
        {"username": USER, "password": PASSWORD},
        {"user": USER, "password": PASSWORD},
        {"email": USER, "password": PASSWORD},
    ]

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            r = sess.post(LOGIN_URL, json=payload, timeout=TIMEOUT)
        except requests.exceptions.RequestException as e:
            print(f"[LOGIN] Falha de rede na tentativa {attempt}/{MAX_RETRIES}: {e}")
            time.sleep(RETRY_BACKOFF ** attempt)
            continue

        if r.status_code == 200:
            try:
                data = r.json()
            except ValueError:
                log_resp(r, prefix="[LOGIN] ")
                raise RuntimeError("[LOGIN] Resposta não-JSON, impossível obter token.")

            token = data.get("token") or data.get("access_token") or data.get("accessToken") or data.get("jwt")
            if not token:
                print("[LOGIN] Token não encontrado nas chaves conhecidas. Chaves recebidas:", list(data.keys()))
                log_resp(r, prefix="[LOGIN] ")
                raise RuntimeError("[LOGIN] Token ausente na resposta.")
            return token

        log_resp(r, prefix=f"[LOGIN] Tentativa {attempt}/{MAX_RETRIES} ")
        if r.status_code in (429, 500, 502, 503, 504):
            time.sleep(RETRY_BACKOFF ** attempt)
            continue

        if r.status_code in (400, 401) and attempt == 1:
            print("[LOGIN] Tentando variações de payload (username/email)...")
            for alt in alt_payloads:
                try:
                    r2 = sess.post(LOGIN_URL, json=alt, timeout=TIMEOUT)
                except requests.exceptions.RequestException as e:
                    print(f"[LOGIN] Falha de rede com payload alternativo: {e}")
                    continue
                if r2.status_code == 200:
                    try:
                        data2 = r2.json()
                    except ValueError:
                        log_resp(r2, prefix="[LOGIN/ALT] ")
                        raise RuntimeError("[LOGIN/ALT] Resposta não-JSON.")
                    token = data2.get("token") or data2.get("access_token") or data2.get("accessToken") or data2.get("jwt")
                    if token:
                        print("[LOGIN] Autenticado com payload alternativo:", alt.keys())
                        return token
                    else:
                        log_resp(r2, prefix="[LOGIN/ALT] ")
                        raise RuntimeError("[LOGIN/ALT] Token ausente.")
                else:
                    log_resp(r2, prefix="[LOGIN/ALT] ")

        raise RuntimeError(f"[LOGIN] Erro ao autenticar. Status={r.status_code}. Veja logs acima.")

    raise RuntimeError("[LOGIN] Não foi possível autenticar após múltiplas tentativas.")


# ============================================
# CHAMADA GENÉRICA DE API COM RETRY
# ============================================

def chamar_api(method: str, endpoint: str, token: str,
               params: Optional[Dict[str, Any]] = None,
               body: Optional[Dict[str, Any]] = None,
               session: Optional[requests.Session] = None) -> Optional[Dict[str, Any]]:
    """
    Faz chamada GET/POST com token, timeout, retry para erros transitórios,
    e logs detalhados de falha.
    """
    sess = session or requests.Session()
    headers = {"Authorization": f"Bearer {token}"}
    url = f"{API_BASE.rstrip('/')}/{endpoint.lstrip('/')}"

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            if method.upper() == "GET":
                r = sess.get(url, headers=headers, params=params, timeout=TIMEOUT)
            else:
                r = sess.post(url, headers=headers, json=body, timeout=TIMEOUT)
        except requests.exceptions.RequestException as e:
            print(f"[API] Falha de rede em {method} {url} (tentativa {attempt}/{MAX_RETRIES}): {e}")
            time.sleep(RETRY_BACKOFF ** attempt)
            continue

        if r.status_code == 200:
            try:
                data = r.json()
            except ValueError:
                log_resp(r, prefix="[API] ")
                print("[API] Resposta não-JSON; retornando None.")
                return None
            time.sleep(RATE_LIMIT_SLEEP)
            return data

        log_resp(r, prefix=f"[API] {method} {endpoint} tentativa {attempt}/{MAX_RETRIES} ")

        if r.status_code in (429, 500, 502, 503, 504):
            time.sleep(RETRY_BACKOFF ** attempt)
            continue

        return None

    print(f"[API] Desistindo de {method} {endpoint} após {MAX_RETRIES} tentativas.")
    return None


# ============================================
# COLETA DE EQUIPAMENTOS
# ============================================

def coletar_equipamentos(token: str, session: Optional[requests.Session] = None) -> Optional[List[Dict[str, Any]]]:
    """
    Retorna lista de equipamentos ativos. Se a API devolver objeto, tenta extrair 'items'.
    """
    params = {"ativo": "true"}
    dados = chamar_api("GET", "equipamento/list", token, params=params, session=session)
    if not dados:
        return None

    if isinstance(dados, list):
        return dados

    if isinstance(dados, dict) and "items" in dados:
        return dados["items"]

    return [dados]


# ============================================
# COLETA DE AMOSTRAS (sampleResult/search) COM PERÍODO E PAGINAÇÃO
# ============================================

def coletar_amostras(token: str, session: Optional[requests.Session] = None) -> List[Dict[str, Any]]:
    """
    Coleta amostras dos últimos PERIODO_DIAS até a data atual, varrendo todas as páginas.
    Retorna uma lista consolidada de itens.
    """
    data_fim = datetime.now().strftime("%Y-%m-%d")
    data_inicio = (datetime.now() - timedelta(days=PERIODO_DIAS)).strftime("%Y-%m-%d")

    print(f"[Amostras] Coletando dados de {data_inicio} até {data_fim}")

    page = 1
    page_size = 500
    todos_itens: List[Dict[str, Any]] = []

    while True:
        body = {
            "readingStatus": None,   # busca tudo
            "markRead": True,        # marcar como lido
            "startDate": data_inicio,
            "endDate": data_fim,
            "page": page,
            "pageSize": page_size
        }

        dados = chamar_api("POST", "sampleResult/search", token, body=body, session=session)
        if not dados:
            # Sem dados ou erro não transitório
            break

        # Normalmente a API retorna {items: [...], total: X, page: Y}
        if isinstance(dados, dict) and isinstance(dados.get("items"), list):
            itens = dados["items"]
        elif isinstance(dados, list):
            itens = dados
        else:
            itens = [dados]

        todos_itens.extend(itens)
        print(f"[Amostras] Página {page}: coletados {len(itens)} itens (total acumulado: {len(todos_itens)})")

        # Heurística de término: se retornou menos que page_size, provavelmente acabou
        if len(itens) < page_size:
            break

        page += 1
        time.sleep(1)  # pequena pausa entre páginas

    return todos_itens


# ============================================
# SALVAR CSV
# ============================================

def salvar_csv(dados: Any, nome: str) -> None:
    """
    Salva lista/dict em CSV com UTF-8 na pasta configurada.
    """
    ensure_dir(PASTA_SAIDA)
    caminho = os.path.join(PASTA_SAIDA, nome)

    if isinstance(dados, list):
        df = pd.DataFrame(dados)
    elif isinstance(dados, dict):
        df = pd.DataFrame([dados])
    else:
        raise ValueError("Formato de dados não suportado para CSV.")

    # Exporta CSV (excel-friendly)
    df.to_csv(caminho, index=False, encoding=CSV_ENCODING, sep=CSV_SEPARADOR)
    print(f"Arquivo salvo: {caminho}")


# ============================================
# EXECUÇÃO PRINCIPAL
# ============================================

def executar_rotina():
    print("Iniciando coleta:", datetime.now())

    session = requests.Session()
    token = autenticar(session=session)

    # Equipamentos
    equipamentos = coletar_equipamentos(token, session=session)
    if equipamentos:
        salvar_csv(equipamentos, f"equipamentos_{datetime.now().date()}.csv")
    else:
        print("[Equipamentos] Nenhum dado retornado.")

    # Amostras (todas as páginas dos últimos PERIODO_DIAS)
    amostras = coletar_amostras(token, session=session)
    if amostras:
        salvar_csv(amostras, f"amostras_{datetime.now().date()}.csv")
    else:
        print("[Amostras] Nenhum dado retornado.")

    print("Finalizado:", datetime.now())


# ============================================
# MAIN
# ============================================

if __name__ == "__main__":
    ensure_dir(PASTA_SAIDA)
    try:
        executar_rotina()
    except Exception as e:
        print("Falha na execução:", e)


Iniciando coleta: 2026-01-13 16:36:45.787751
[LOGIN] Tentativa 1/3 Status: 400 Bad Request | Content-Type: 
[LOGIN] Tentativa 1/3 Body (até 1000 chars): 
[LOGIN] Tentativa 1/3 Headers: {'Date': 'Tue, 13 Jan 2026 16:36:46 GMT'}
[LOGIN] Tentando variações de payload (username/email)...
[LOGIN] Autenticado com payload alternativo: dict_keys(['username', 'password'])
[API] GET equipamento/list tentativa 1/3 Status: 405 Method Not Allowed | Content-Type: application/xml;charset=UTF-8
[API] GET equipamento/list tentativa 1/3 Body (até 1000 chars): <Map><timestamp>1768322207225</timestamp><status>405</status><error>Method Not Allowed</error><path>/api/v1/equipamento/list</path></Map>
[API] GET equipamento/list tentativa 1/3 Headers: {'Date': 'Tue, 13 Jan 2026 16:36:46 GMT'}
[Equipamentos] Nenhum dado retornado.
[Amostras] Coletando dados de 2025-01-13 até 2026-01-13
[Amostras] Página 1: coletados 1 itens (total acumulado: 1)
Arquivo salvo: /content/s360_export/amostras_2026-01-13.csv
Finaliza

In [4]:

import os
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
from typing import Optional, Dict, Any, List, Tuple

# ============================================
# CONFIGURAÇÕES DA API
# ============================================

LOGIN_URL = "https://integration.s360web.com/api/login"
API_BASE = "https://integration.s360web.com/api/v1"

USER = os.getenv("S360_USER", "integracao.komatsu")
PASSWORD = os.getenv("S360_PASSWORD", "o8sqfl5h")

PASTA_SAIDA = "/content/s360_export/"
PERIODO_DIAS = 365         # backfill de 1 ano
CHUNK_DIAS = 31            # janelas mensais para cobrir 100%
PAGE_SIZE = 500            # tamanhos grandes (ver limite no portal)
TIMEOUT = (10, 60)         # (connect, read)
MAX_RETRIES = 3
RETRY_BACKOFF = 2          # base exponencial
SLEEP_ENTRE_CHAMADAS = 25  # orientação S360

CSV_SEPARADOR = ","
CSV_ENCODING = "utf-8-sig"

# ============================================
# UTILITÁRIOS
# ============================================

def ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)

def log_resp(r: requests.Response, prefix: str = "") -> None:
    ct = r.headers.get("Content-Type", "")
    print(f"{prefix}Status: {r.status_code} {r.reason} | Content-Type: {ct}")
    body = r.text or ""
    print(f"{prefix}Body (até 1000 chars): {body[:1000]}")
    importantes = ['date', 'server', 'x-request-id', 'x-correlation-id']
    print(f"{prefix}Headers: { {k: v for k, v in r.headers.items() if k.lower() in importantes} }")

# ============================================
# AUTENTICAÇÃO
# ============================================

def autenticar(session: Optional[requests.Session] = None) -> str:
    sess = session or requests.Session()
    payloads = [
        {"login": USER, "password": PASSWORD},    # já funciona no seu ambiente
        {"username": USER, "password": PASSWORD}, # variação vista no dev portal
        {"email": USER, "password": PASSWORD},
    ]
    for attempt in range(1, MAX_RETRIES + 1):
        for payload in payloads:
            try:
                r = sess.post(LOGIN_URL, json=payload, timeout=TIMEOUT)
            except requests.exceptions.RequestException as e:
                print(f"[LOGIN] Falha de rede {attempt}/{MAX_RETRIES}: {e}")
                time.sleep(RETRY_BACKOFF ** attempt)
                continue

            if r.status_code == 200:
                try:
                    data = r.json()
                except ValueError:
                    log_resp(r, prefix="[LOGIN] ")
                    raise RuntimeError("[LOGIN] Resposta não-JSON.")
                token = data.get("token") or data.get("access_token") or data.get("accessToken") or data.get("jwt")
                if token:
                    print("[LOGIN] Autenticado com payload:", list(payload.keys()))
                    return token
                log_resp(r, prefix="[LOGIN] ")
                raise RuntimeError("[LOGIN] Token ausente na resposta.")
            else:
                log_resp(r, prefix="[LOGIN] ")

        # 429/5xx: backoff
        time.sleep(RETRY_BACKOFF ** attempt)

    raise RuntimeError("[LOGIN] Não foi possível autenticar após múltiplas tentativas.")

# ============================================
# CHAMADAS DE API (GET/POST) COM RETRY
# ============================================

def chamar_api(method: str, endpoint: str, token: str,
               params: Optional[Dict[str, Any]] = None,
               body: Optional[Dict[str, Any]] = None,
               session: Optional[requests.Session] = None) -> Optional[Dict[str, Any]]:
    sess = session or requests.Session()
    headers = {"Authorization": f"Bearer {token}"}
    url = f"{API_BASE.rstrip('/')}/{endpoint.lstrip('/')}"

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            if method.upper() == "GET":
                r = sess.get(url, headers=headers, params=params, timeout=TIMEOUT)
            else:
                r = sess.post(url, headers=headers, json=body, timeout=TIMEOUT)
        except requests.exceptions.RequestException as e:
            print(f"[API] Falha de rede em {method} {url} ({attempt}/{MAX_RETRIES}): {e}")
            time.sleep(RETRY_BACKOFF ** attempt)
            continue

        if r.status_code == 200:
            try:
                data = r.json()
            except ValueError:
                log_resp(r, prefix="[API] ")
                print("[API] Resposta não-JSON; retornando None.")
                return None
            # sleep entre chamadas (orientação S360)
            time.sleep(SLEEP_ENTRE_CHAMADAS)
            return data

        log_resp(r, prefix=f"[API] {method} {endpoint} tentativa {attempt}/{MAX_RETRIES} ")
        if r.status_code in (429, 500, 502, 503, 504):
            time.sleep(RETRY_BACKOFF ** attempt)
            continue

        return None

    print(f"[API] Desistindo de {method} {endpoint} após {MAX_RETRIES} tentativas.")
    return None

# ============================================
# EQUIPAMENTOS (ativos)
# ============================================

def coletar_equipamentos(token: str, session: Optional[requests.Session] = None) -> Optional[List[Dict[str, Any]]]:
    params = {"ativo": "true"}
    dados = chamar_api("GET", "equipamento/list", token, params=params, session=session)
    if not dados:
        return None
    if isinstance(dados, list):
        return dados
    if isinstance(dados, dict) and "items" in dados:
        return dados["items"]
    return [dados]

# ============================================
# AMOSTRAS: RESULTADOS (sampleResult/search) com CHUNK + paginação
# ============================================

def gerar_janelas_data(periodo_dias: int, chunk_dias: int) -> List[Tuple[str, str]]:
    """Gera janelas [startDate, endDate] em ISO YYYY-MM-DD, do passado até hoje."""
    hoje = datetime.now().date()
    inicio = hoje - timedelta(days=periodo_dias)
    janelas = []
    cursor = inicio
    while cursor <= hoje:
        fim_chunk = min(cursor + timedelta(days=chunk_dias - 1), hoje)
        janelas.append((cursor.strftime("%Y-%m-%d"), fim_chunk.strftime("%Y-%m-%d")))
        cursor = fim_chunk + timedelta(days=1)
    return janelas

def coletar_resultados(token: str, periodo_dias: int = PERIODO_DIAS,
                       chunk_dias: int = CHUNK_DIAS,
                       session: Optional[requests.Session] = None) -> List[Dict[str, Any]]:
    """
    Coleta 100% dos resultados em janelas de 'chunk_dias', paginando até o fim.
    Usa readingStatus=null (com filtro de data) e markRead=null (não marca como lidas).
    """
    todos: List[Dict[str, Any]] = []
    janelas = gerar_janelas_data(periodo_dias, chunk_dias)
    print(f"[Resultados] Janelas geradas: {len(janelas)} (de {janelas[0][0]} até {janelas[-1][1]})")

    for (data_inicio, data_fim) in janelas:
        page = 1
        print(f"[Resultados] Janela {data_inicio} → {data_fim}")
        while True:
            body = {
                "readingStatus": None,   # null: todos (com filtro de data)
                "markRead": None,        # null: não marcar como lidas no backfill
                "startDate": data_inicio,
                "endDate": data_fim,
                "page": page,
                "pageSize": PAGE_SIZE,
            }
            dados = chamar_api("POST", "sampleResult/search", token, body=body, session=session)
            if not dados:
                print(f"[Resultados] Sem dados ou falha na janela {data_inicio} → {data_fim}, página {page}.")
                break

            if isinstance(dados, dict) and isinstance(dados.get("items"), list):
                itens = dados["items"]
            elif isinstance(dados, list):
                itens = dados
            else:
                itens = [dados]

            qtd = len(itens)
            todos.extend(itens)
            print(f"[Resultados] {data_inicio} → {data_fim} | página {page} | itens {qtd} | acumulado {len(todos)}")

            if qtd < PAGE_SIZE:
                break  # última página da janela
            page += 1

    return todos

# ============================================
# SALVAR CSV
# ============================================

def salvar_csv(dados: Any, nome: str) -> None:
    ensure_dir(PASTA_SAIDA)
    caminho = os.path.join(PASTA_SAIDA, nome)
    if isinstance(dados, list):
        df = pd.DataFrame(dados)
    elif isinstance(dados, dict):
        df = pd.DataFrame([dados])
    else:
        raise ValueError("Formato de dados não suportado para CSV.")
    df.to_csv(caminho, index=False, encoding=CSV_ENCODING, sep=CSV_SEPARADOR)
    print(f"Arquivo salvo: {caminho}")

# ============================================
# EXECUÇÃO PRINCIPAL
# ============================================

def executar_rotina():
    print("Iniciando coleta:", datetime.now())
    session = requests.Session()
    token = autenticar(session=session)

    # Equipamentos (ativos)
    equipamentos = coletar_equipamentos(token, session=session)
    if equipamentos:
        salvar_csv(equipamentos, f"equipamentos_{datetime.now().date()}.csv")
    else:
        print("[Equipamentos] Nenhum dado retornado.")

    # Resultados de amostra (100% do último ano em janelas + paginação)
    resultados = coletar_resultados(token, periodo_dias=PERIODO_DIAS, chunk_dias=CHUNK_DIAS, session=session)
    if resultados:
        salvar_csv(resultados, f"resultados_{datetime.now().date()}.csv")
    else:
        print("[Resultados] Nenhum dado retornado.")

    print("Finalizado:", datetime.now())

if __name__ == "__main__":
    ensure_dir(PASTA_SAIDA)
    try:
        executar_rotina()
    except Exception as e:
        print("Falha na execução:", e)


Iniciando coleta: 2026-01-13 17:34:22.389842
[LOGIN] Status: 400 Bad Request | Content-Type: 
[LOGIN] Body (até 1000 chars): 
[LOGIN] Headers: {'Date': 'Tue, 13 Jan 2026 17:34:23 GMT'}
[LOGIN] Autenticado com payload: ['username', 'password']
[API] GET equipamento/list tentativa 1/3 Status: 405 Method Not Allowed | Content-Type: application/xml;charset=UTF-8
[API] GET equipamento/list tentativa 1/3 Body (até 1000 chars): <Map><timestamp>1768325663774</timestamp><status>405</status><error>Method Not Allowed</error><path>/api/v1/equipamento/list</path></Map>
[API] GET equipamento/list tentativa 1/3 Headers: {'Date': 'Tue, 13 Jan 2026 17:34:23 GMT'}
[Equipamentos] Nenhum dado retornado.
[Resultados] Janelas geradas: 12 (de 2025-01-13 até 2026-01-13)
[Resultados] Janela 2025-01-13 → 2025-02-12
[Resultados] 2025-01-13 → 2025-02-12 | página 1 | itens 1 | acumulado 1
[Resultados] Janela 2025-02-13 → 2025-03-15
[Resultados] 2025-02-13 → 2025-03-15 | página 1 | itens 1 | acumulado 2
[Resultados

In [1]:

import os
import re
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
from typing import Optional, Dict, Any, List, Tuple

# ============================================
# CONFIGURAÇÕES DA API
# ============================================

LOGIN_URL = "https://integration.s360web.com/api/login"
API_BASE = "https://integration.s360web.com/api/v1"

USER = os.getenv("S360_USER", "integracao.komatsu")
PASSWORD = os.getenv("S360_PASSWORD", "o8sqfl5h")

PASTA_SAIDA = "/content/s360_export/"
PERIODO_DIAS = 365         # backfill de 1 ano
CHUNK_DIAS = 31            # janelas mensais para cobrir 100%
PAGE_SIZE = 500            # tamanho de página
TIMEOUT = (10, 60)         # (connect, read)
MAX_RETRIES = 3
RETRY_BACKOFF = 2          # base exponencial
SLEEP_ENTRE_CHAMADAS = 25  # orientação S360

CSV_SEPARADOR = ","
CSV_ENCODING = "utf-8-sig"

# ============================================
# UTILITÁRIOS
# ============================================

def ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)

def log_resp(r: requests.Response, prefix: str = "") -> None:
    ct = r.headers.get("Content-Type", "")
    print(f"{prefix}Status: {r.status_code} {r.reason} | Content-Type: {ct}")
    body = r.text or ""
    print(f"{prefix}Body (até 1000 chars): {body[:1000]}")
    importantes = ['date', 'server', 'x-request-id', 'x-correlation-id']
    print(f"{prefix}Headers: { {k: v for k, v in r.headers.items() if k.lower() in importantes} }")

def get(o: Dict[str, Any], path: str, default: Any = None) -> Any:
    """
    Acesso seguro a objetos aninhados usando caminho com pontos.
    Ex.: get(sample, 'equipment.model')
    """
    cur = o
    for p in path.split('.'):
        if cur is None:
            return default
        if isinstance(cur, dict):
            cur = cur.get(p, default)
        else:
            return default
    return cur

def normalize_decimal(value: Any) -> Tuple[Optional[float], str, bool]:
    """
    Tenta converter strings como '24,921660', '11,80', '1,817' em float com ponto.
    Mantém o texto original em 'clean' e informa se é numérico via 'is_num'.
    Não força conversão para tokens como '<1', '23/21/15', '-', 'I.P'.
    """
    if value is None:
        return None, "", False
    if isinstance(value, (int, float)):
        return float(value), str(value), True
    s = str(value).strip()
    # Casos não-numéricos óbvios
    if s in {"-", "", "I.P"} or "/" in s or s.startswith("<") or s.startswith(">"):
        return None, s, False
    # Troca vírgula por ponto e remove espaços
    s2 = s.replace(",", ".")
    # Mantém sinais e ponto; remove outros caracteres não numéricos
    s2 = re.sub(r"[^0-9\.\-eE]", "", s2)
    try:
        num = float(s2)
        return num, s, True
    except ValueError:
        return None, s, False

def join_names(items: List[Dict[str, Any]], field_path: str = "name", sep: str = " | ") -> str:
    """
    Junta nomes de uma lista de objetos (e.g., testPackages) em uma string.
    """
    vals = []
    for it in items or []:
        v = get(it, field_path)
        if v:
            vals.append(str(v))
    return sep.join(vals)

# ============================================
# AUTENTICAÇÃO
# ============================================

def autenticar(session: Optional[requests.Session] = None) -> str:
    sess = session or requests.Session()
    payloads = [
        {"login": USER, "password": PASSWORD},    # funciona no seu ambiente
        {"username": USER, "password": PASSWORD}, # variação vista no dev portal
        {"email": USER, "password": PASSWORD},
    ]
    for attempt in range(1, MAX_RETRIES + 1):
        for payload in payloads:
            try:
                r = sess.post(LOGIN_URL, json=payload, timeout=TIMEOUT)
            except requests.exceptions.RequestException as e:
                print(f"[LOGIN] Falha de rede {attempt}/{MAX_RETRIES}: {e}")
                time.sleep(RETRY_BACKOFF ** attempt)
                continue

            if r.status_code == 200:
                try:
                    data = r.json()
                except ValueError:
                    log_resp(r, prefix="[LOGIN] ")
                    raise RuntimeError("[LOGIN] Resposta não-JSON.")
                token = data.get("token") or data.get("access_token") or data.get("accessToken") or data.get("jwt")
                if token:
                    print("[LOGIN] Autenticado com payload:", list(payload.keys()))
                    return token
                log_resp(r, prefix="[LOGIN] ")
                raise RuntimeError("[LOGIN] Token ausente na resposta.")
            else:
                log_resp(r, prefix="[LOGIN] ")

        time.sleep(RETRY_BACKOFF ** attempt)

    raise RuntimeError("[LOGIN] Não foi possível autenticar após múltiplas tentativas.")

# ============================================
# CHAMADAS DE API (GET/POST) COM RETRY
# ============================================

def chamar_api(method: str, endpoint: str, token: str,
               params: Optional[Dict[str, Any]] = None,
               body: Optional[Dict[str, Any]] = None,
               session: Optional[requests.Session] = None) -> Optional[Dict[str, Any]]:
    sess = session or requests.Session()
    headers = {"Authorization": f"Bearer {token}"}
    url = f"{API_BASE.rstrip('/')}/{endpoint.lstrip('/')}"

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            if method.upper() == "GET":
                r = sess.get(url, headers=headers, params=params, timeout=TIMEOUT)
            else:
                r = sess.post(url, headers=headers, json=body, timeout=TIMEOUT)
        except requests.exceptions.RequestException as e:
            print(f"[API] Falha de rede em {method} {url} ({attempt}/{MAX_RETRIES}): {e}")
            time.sleep(RETRY_BACKOFF ** attempt)
            continue

        if r.status_code == 200:
            try:
                data = r.json()
            except ValueError:
                log_resp(r, prefix="[API] ")
                print("[API] Resposta não-JSON; retornando None.")
                return None
            time.sleep(SLEEP_ENTRE_CHAMADAS)  # orientação S360
            return data

        log_resp(r, prefix=f"[API] {method} {endpoint} tentativa {attempt}/{MAX_RETRIES} ")
        if r.status_code in (429, 500, 502, 503, 504):
            time.sleep(RETRY_BACKOFF ** attempt)
            continue

        return None

    print(f"[API] Desistindo de {method} {endpoint} após {MAX_RETRIES} tentativas.")
    return None

# ============================================
# EQUIPAMENTOS (ativos)
# ============================================

def coletar_equipamentos(token: str, session: Optional[requests.Session] = None) -> Optional[List[Dict[str, Any]]]:
    params = {"ativo": "true"}
    dados = chamar_api("GET", "equipamento/list", token, params=params, session=session)
    if not dados:
        return None
    if isinstance(dados, list):
        return dados
    if isinstance(dados, dict) and "items" in dados:
        return dados["items"]
    return [dados]

# ============================================
# RESULTADOS (sampleResult/search) com CHUNK + paginação
# ============================================

def gerar_janelas_data(periodo_dias: int, chunk_dias: int) -> List[Tuple[str, str]]:
    hoje = datetime.now().date()
    inicio = hoje - timedelta(days=periodo_dias)
    janelas = []
    cursor = inicio
    while cursor <= hoje:
        fim_chunk = min(cursor + timedelta(days=chunk_dias - 1), hoje)
        janelas.append((cursor.strftime("%Y-%m-%d"), fim_chunk.strftime("%Y-%m-%d")))
        cursor = fim_chunk + timedelta(days=1)
    return janelas

def coletar_resultados(token: str, periodo_dias: int = PERIODO_DIAS,
                       chunk_dias: int = CHUNK_DIAS,
                       session: Optional[requests.Session] = None) -> List[Dict[str, Any]]:
    todos: List[Dict[str, Any]] = []
    janelas = gerar_janelas_data(periodo_dias, chunk_dias)
    print(f"[Resultados] Janelas geradas: {len(janelas)} (de {janelas[0][0]} até {janelas[-1][1]})")

    for (data_inicio, data_fim) in janelas:
        page = 1
        print(f"[Resultados] Janela {data_inicio} → {data_fim}")
        while True:
            body = {
                "readingStatus": None,   # null: todos (com filtro de data)
                "markRead": None,        # null: não marcar como lidas no backfill
                "startDate": data_inicio,
                "endDate": data_fim,
                "page": page,
                "pageSize": PAGE_SIZE,
            }
            dados = chamar_api("POST", "sampleResult/search", token, body=body, session=session)
            if not dados:
                print(f"[Resultados] Sem dados ou falha na janela {data_inicio} → {data_fim}, página {page}.")
                break

            if isinstance(dados, dict) and isinstance(dados.get("items"), list):
                itens = dados["items"]
            elif isinstance(dados, list):
                itens = dados
            else:
                itens = [dados]

            qtd = len(itens)
            todos.extend(itens)
            print(f"[Resultados] {data_inicio} → {data_fim} | página {page} | itens {qtd} | acumulado {len(todos)}")

            if qtd < PAGE_SIZE:
                break
            page += 1

    return todos

# ============================================
# ETL: FLATTEN (1 linha = 1 teste por amostra)
# ============================================

def flatten_resultados(resultados: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Transforma cada amostra em N linhas (uma por teste), com metadados completos.
    Converte resultValue para número (quando possível), mantendo original.
    """
    linhas: List[Dict[str, Any]] = []

    for s in resultados or []:
        # Metadados de nível amostra
        sample_id         = s.get("id")
        sample_number     = s.get("sampleNumber")
        external_code     = s.get("externalCode")
        result_date       = s.get("resultDate")
        reading_status    = s.get("readingStatus")

        # validResult (status geral, avaliação e ação de inspeção)
        vr_status         = get(s, "validResult.resultStatus")
        vr_eval           = get(s, "validResult.evaluation")
        vr_action         = get(s, "validResult.inspectionAction")

        # Metadados de equipamento/cliente/site/compartimento
        equip_model       = get(s, "equipment.model")
        equip_serial      = get(s, "equipment.serial")
        equip_tag         = get(s, "equipment.tag")
        equip_family      = get(s, "equipment.family.name")
        equip_maker       = get(s, "equipment.maker.name")
        site_name         = get(s, "site.name") or get(s, "equipment.site.name")
        site_ext_code     = get(s, "site.externalCode") or get(s, "equipment.site.externalCode")
        customer_name     = get(s, "customer.name")
        customer_id       = get(s, "customer.id")
        operation_name    = get(s, "operation.name")
        lab_name          = get(s, "laboratory.name")
        payment_terms     = s.get("paymentTerms")

        comp_name         = get(s, "compartment.name") or get(s, "collectionData.compartmentName")
        comp_type         = get(s, "compartment.type.name") or get(s, "collectionData.compartmentType.name")
        comp_volume       = get(s, "compartment.volume")
        coll_date_sampled = get(s, "collectionData.dateSampled")
        coll_reg_date     = get(s, "collectionData.registrationDate")
        coll_time_type    = get(s, "collectionData.timeType")
        coll_fluid_time   = get(s, "collectionData.fluidTime")
        coll_equipt_time  = get(s, "collectionData.equipmentTime")
        oil_viscosity     = get(s, "collectionData.oil.viscosity.name")
        oil_manufacturer  = get(s, "collectionData.oil.manufacturer.name")
        oil_changed       = get(s, "collectionData.oilChanged")

        test_packages     = join_names(s.get("testPackages") or [], "name")

        # Lista de testes por amostra
        tests = s.get("testResults") or []
        if not tests:
            # Ainda assim, registramos uma linha "sem teste" para rastreabilidade
            linhas.append({
                "sampleId": sample_id,
                "sampleNumber": sample_number,
                "externalCode": external_code,
                "resultDate": result_date,
                "readingStatus": reading_status,
                "validResultStatus": vr_status,
                "validResultEvaluation": vr_eval,
                "validResultInspectionAction": vr_action,
                "equipmentModel": equip_model,
                "equipmentSerial": equip_serial,
                "equipmentTag": equip_tag,
                "equipmentFamily": equip_family,
                "equipmentMaker": equip_maker,
                "siteName": site_name,
                "siteExternalCode": site_ext_code,
                "customerName": customer_name,
                "customerId": customer_id,
                "operationName": operation_name,
                "laboratoryName": lab_name,
                "paymentTerms": payment_terms,
                "compartmentName": comp_name,
                "compartmentType": comp_type,
                "compartmentVolume": comp_volume,
                "collectionDateSampled": coll_date_sampled,
                "collectionRegistrationDate": coll_reg_date,
                "collectionTimeType": coll_time_type,
                "collectionFluidTime": coll_fluid_time,
                "collectionEquipmentTime": coll_equipt_time,
                "oilViscosity": oil_viscosity,
                "oilManufacturer": oil_manufacturer,
                "oilChanged": oil_changed,
                "testGroup": None,
                "testName": None,
                "testAbbreviation": None,
                "testUnitOfMeasure": None,
                "testMethod": None,
                "testValueType": None,
                "testOrder": None,
                "resultValue_raw": None,
                "resultValue_num": None,
                "resultValue_is_numeric": False,
                "resultStatus": None,
                "testPackages": test_packages,
            })
            continue

        for tr in tests:
            test_group       = get(tr, "test.testGroup.name")
            test_name        = get(tr, "test.translation.name")
            test_abbrev      = get(tr, "test.translation.abbreviation")
            test_uom         = get(tr, "test.translation.unitOfMeasure")
            test_method      = get(tr, "test.translation.method")
            test_value_type  = get(tr, "test.valueType")
            test_order       = get(tr, "test.order")
            result_status    = tr.get("resultStatus")
            result_value     = tr.get("resultValue")

            num, raw, is_num = normalize_decimal(result_value)

            linhas.append({
                # Amostra
                "sampleId": sample_id,
                "sampleNumber": sample_number,
                "externalCode": external_code,
                "resultDate": result_date,
                "readingStatus": reading_status,
                "validResultStatus": vr_status,
                "validResultEvaluation": vr_eval,
                "validResultInspectionAction": vr_action,

                # Equip/cliente/site
                "equipmentModel": equip_model,
                "equipmentSerial": equip_serial,
                "equipmentTag": equip_tag,
                "equipmentFamily": equip_family,
                "equipmentMaker": equip_maker,
                "siteName": site_name,
                "siteExternalCode": site_ext_code,
                "customerName": customer_name,
                "customerId": customer_id,
                "operationName": operation_name,
                "laboratoryName": lab_name,
                "paymentTerms": payment_terms,

                # Compartimento / coleta / óleo
                "compartmentName": comp_name,
                "compartmentType": comp_type,
                "compartmentVolume": comp_volume,
                "collectionDateSampled": coll_date_sampled,
                "collectionRegistrationDate": coll_reg_date,
                "collectionTimeType": coll_time_type,
                "collectionFluidTime": coll_fluid_time,
                "collectionEquipmentTime": coll_equipt_time,
                "oilViscosity": oil_viscosity,
                "oilManufacturer": oil_manufacturer,
                "oilChanged": oil_changed,

                # Teste
                "testGroup": test_group,
                "testName": test_name,
                "testAbbreviation": test_abbrev,
                "testUnitOfMeasure": test_uom,
                "testMethod": test_method,
                "testValueType": test_value_type,
                "testOrder": test_order,
                "resultValue_raw": raw,
                "resultValue_num": num,
                "resultValue_is_numeric": is_num,
                "resultStatus": result_status,

                # Pacotes de teste (lista)
                "testPackages": test_packages,
            })

    return linhas

# ============================================
# SALVAR CSV
# ============================================

def salvar_csv(dados: Any, nome: str) -> None:
    ensure_dir(PASTA_SAIDA)
    caminho = os.path.join(PASTA_SAIDA, nome)
    if isinstance(dados, list):
        df = pd.DataFrame(dados)
    elif isinstance(dados, dict):
        df = pd.DataFrame([dados])
    else:
        raise ValueError("Formato de dados não suportado para CSV.")
    df.to_csv(caminho, index=False, encoding=CSV_ENCODING, sep=CSV_SEPARADOR)
    print(f"Arquivo salvo: {caminho}")

# ============================================
# EXECUÇÃO PRINCIPAL
# ============================================

def executar_rotina():
    print("Iniciando coleta:", datetime.now())
    session = requests.Session()
    token = autenticar(session=session)

    # Equipamentos (ativos)
    equipamentos = coletar_equipamentos(token, session=session)
    if equipamentos:
        salvar_csv(equipamentos, f"equipamentos_{datetime.now().date()}.csv")
    else:
        print("[Equipamentos] Nenhum dado retornado.")

    # Resultados (100% do último ano em janelas + paginação)
    resultados = coletar_resultados(token, periodo_dias=PERIODO_DIAS, chunk_dias=CHUNK_DIAS, session=session)
    if resultados:
        # Salva o bruto (como referência/origem)
        salvar_csv(resultados, f"resultados_{datetime.now().date()}.csv")

        # ETL: flatten → uma linha por teste (tabular para BI/Excel)
        resultados_flat = flatten_resultados(resultados)
        if resultados_flat:
            salvar_csv(resultados_flat, f"resultados_flat_{datetime.now().date()}.csv")
        else:
            print("[Flatten] Nenhuma linha gerada (lista de testes vazia em todas as amostras).")
    else:
        print("[Resultados] Nenhum dado retornado.")

    print("Finalizado:", datetime.now())

# ============================================
# MAIN
# ============================================

if __name__ == "__main__":
    ensure_dir(PASTA_SAIDA)
    try:
        executar_rotina()
    except Exception as e:
        print("Falha na execução:", e)


Iniciando coleta: 2026-01-13 22:45:59.266049
[LOGIN] Status: 400 Bad Request | Content-Type: 
[LOGIN] Body (até 1000 chars): 
[LOGIN] Headers: {'Date': 'Tue, 13 Jan 2026 22:45:59 GMT'}
[LOGIN] Autenticado com payload: ['username', 'password']
[API] GET equipamento/list tentativa 1/3 Status: 405 Method Not Allowed | Content-Type: application/xml;charset=UTF-8
[API] GET equipamento/list tentativa 1/3 Body (até 1000 chars): <Map><timestamp>1768344359654</timestamp><status>405</status><error>Method Not Allowed</error><path>/api/v1/equipamento/list</path></Map>
[API] GET equipamento/list tentativa 1/3 Headers: {'Date': 'Tue, 13 Jan 2026 22:45:59 GMT'}
[Equipamentos] Nenhum dado retornado.
[Resultados] Janelas geradas: 12 (de 2025-01-13 até 2026-01-13)
[Resultados] Janela 2025-01-13 → 2025-02-12
[Resultados] 2025-01-13 → 2025-02-12 | página 1 | itens 1 | acumulado 1
[Resultados] Janela 2025-02-13 → 2025-03-15
[Resultados] 2025-02-13 → 2025-03-15 | página 1 | itens 1 | acumulado 2
[Resultados

In [ ]:

import os
import re
import csv
import sys
import time
import json
import ast
import requests
import pandas as pd
from csv import QUOTE_MINIMAL
from datetime import datetime, timedelta
from typing import Optional, Dict, Any, List, Tuple

# ============================================
# CONFIGURAÇÕES DA API (conforme solicitado)
# ============================================

LOGIN_URL = "https://integration.s360web.com/api/login"
API_BASE  = "https://integration.s360web.com/api/v1"

USER     = os.getenv("S360_USER", "integracao.komatsu")
PASSWORD = os.getenv("S360_PASSWORD", "o8sqfl5h")

PASTA_SAIDA   = "/content/s360_export/"
PERIODO_DIAS  = 365        # backfill de 1 ano
CHUNK_DIAS    = 31         # janelas mensais para cobrir 100%
PAGE_SIZE     = 500        # tamanho de página
TIMEOUT       = (10, 60)   # (connect, read)
MAX_RETRIES   = 3
RETRY_BACKOFF = 2          # base exponencial
SLEEP_ENTRE_CHAMADAS = 25  # orientação S360

# Saída CSV (você pediu vírgula)
CSV_SEPARADOR = ","
CSV_ENCODING  = "utf-8-sig"

# ============================================
# CONFIGURAÇÕES ADICIONAIS
# ============================================

# Ordem opcional de colunas (None = não reordena)
COL_ORDER: Optional[List[str]] = None

# Modo de ingestão: "api" (padrão) ou "csv" (processa arquivo com coluna 'results')
MODE     = os.getenv("MODE", "api").lower().strip()
SRC_FILE = os.getenv("SRC_FILE", "").strip()

# Permitir campos longos (para coluna 'results' quando o modo é csv)
try:
    csv.field_size_limit(sys.maxsize)
except Exception:
    pass

# ============================================
# UTILITÁRIOS
# ============================================

def ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)

def log_resp(r: requests.Response, prefix: str = "") -> None:
    ct = r.headers.get("Content-Type", "")
    print(f"{prefix}Status: {r.status_code} {r.reason} | Content-Type: {ct}")
    body = r.text or ""
    print(f"{prefix}Body (até 1000 chars): {body[:1000]}")
    importantes = ['date', 'server', 'x-request-id', 'x-correlation-id']
    print(f"{prefix}Headers: { {k: v for k, v in r.headers.items() if k.lower() in importantes} }")

def get(o: Dict[str, Any], path: str, default: Any = "") -> Any:
    """
    Acesso seguro 'a.b.c'. Retorna '' por padrão (excel-friendly).
    """
    cur = o
    for p in path.split('.'):
        if cur is None:
            return default
        if isinstance(cur, dict):
            cur = cur.get(p, default)
        else:
            return default
    return cur if cur is not None else default

def normalize_decimal(value: Any) -> Tuple[Optional[float], str, bool]:
    """
    Converte '24,921660' -> 24.92166. Mantém original em 'raw'.
    Não converte tokens: '<1', '23/21/15', '-', 'I.P'.
    """
    if value is None:
        return None, "", False
    if isinstance(value, (int, float)):
        return float(value), str(value), True
    s = str(value).strip()
    if s in {"-", "", "I.P"} or "/" in s or s.startswith("<") or s.startswith(">"):
        return None, s, False
    s2 = s.replace(",", ".")
    s2 = re.sub(r"[^0-9\.\-eE]", "", s2)
    try:
        return float(s2), s, True
    except ValueError:
        return None, s, False

def join_names(items: List[Dict[str, Any]], field_path: str = "name", sep: str = " | ") -> str:
    vals = []
    for it in items or []:
        v = get(it, field_path)
        if v:
            vals.append(str(v))
    return sep.join(vals)

def fix_mojibake(s: Any) -> Any:
    """Corrige acentuação quebrada típica (Ã©, Ã§, etc.) quando aplicável."""
    if not isinstance(s, str):
        return s
    try:
        return s.encode("latin1").decode("utf-8")
    except Exception:
        return s

def aplicar_ordem_colunas(df: pd.DataFrame, col_order: Optional[List[str]] = None) -> pd.DataFrame:
    if not col_order:
        return df
    keep = [c for c in col_order if c in df.columns]
    extra = [c for c in df.columns if c not in keep]
    return df[keep + extra]

def detectar_sep_csv(src_file: str) -> str:
    """
    Detecta automaticamente o delimitador do CSV de entrada (';' ou ',').
    Usa a primeira linha não vazia para decidir.
    """
    with open(src_file, "r", encoding=CSV_ENCODING, errors="replace") as f:
        for line in f:
            ln = line.strip()
            if not ln:
                continue
            # conta ocorrências
            sc = ln.count(";")
            cm = ln.count(",")
            if sc > cm:
                return ";"
            elif cm > sc:
                return ","
            # empate: tenta ';' por padrão (mais comum pt-BR)
            return ";"
    # default seguro
    return ";"

# ============================================
# AUTENTICAÇÃO
# ============================================

def autenticar(session: Optional[requests.Session] = None) -> str:
    sess = session or requests.Session()
    payloads = [
        {"login": USER,     "password": PASSWORD},
        {"username": USER,  "password": PASSWORD},
        {"email": USER,     "password": PASSWORD},
    ]
    for attempt in range(1, MAX_RETRIES + 1):
        for payload in payloads:
            try:
                r = sess.post(LOGIN_URL, json=payload, timeout=TIMEOUT)
            except requests.exceptions.RequestException as e:
                print(f"[LOGIN] Falha de rede {attempt}/{MAX_RETRIES}: {e}")
                time.sleep(RETRY_BACKOFF ** attempt)
                continue

            if r.status_code == 200:
                try:
                    data = r.json()
                except ValueError:
                    log_resp(r, prefix="[LOGIN] ")
                    raise RuntimeError("[LOGIN] Resposta não-JSON.")
                token = data.get("token") or data.get("access_token") \
                        or data.get("accessToken") or data.get("jwt")
                if token:
                    print("[LOGIN] Autenticado com payload:", list(payload.keys()))
                    return token
                log_resp(r, prefix="[LOGIN] ")
                raise RuntimeError("[LOGIN] Token ausente na resposta.")
            else:
                log_resp(r, prefix="[LOGIN] ")
        time.sleep(RETRY_BACKOFF ** attempt)
    raise RuntimeError("[LOGIN] Não foi possível autenticar após múltiplas tentativas.")

# ============================================
# CHAMADAS DE API (GET/POST) COM RETRY
# ============================================

def chamar_api(method: str, endpoint: str, token: str,
               params: Optional[Dict[str, Any]] = None,
               body: Optional[Dict[str, Any]] = None,
               session: Optional[requests.Session] = None) -> Optional[Dict[str, Any]]:
    sess = session or requests.Session()
    headers = {"Authorization": f"Bearer {token}"}
    url = f"{API_BASE.rstrip('/')}/{endpoint.lstrip('/')}"

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            if method.upper() == "GET":
                r = sess.get(url, headers=headers, params=params, timeout=TIMEOUT)
            else:
                r = sess.post(url, headers=headers, json=body, timeout=TIMEOUT)
        except requests.exceptions.RequestException as e:
            print(f"[API] Falha de rede em {method} {url} ({attempt}/{MAX_RETRIES}): {e}")
            time.sleep(RETRY_BACKOFF ** attempt)
            continue

        if r.status_code == 200:
            try:
                data = r.json()
            except ValueError:
                log_resp(r, prefix="[API] ")
                print("[API] Resposta não-JSON; retornando None.")
                return None
            time.sleep(SLEEP_ENTRE_CHAMADAS)
            return data

        log_resp(r, prefix=f"[API] {method} {endpoint} tentativa {attempt}/{MAX_RETRIES} ")
        if r.status_code in (429, 500, 502, 503, 504):
            time.sleep(RETRY_BACKOFF ** attempt)
            continue

        return None

    print(f"[API] Desistindo de {method} {endpoint} após {MAX_RETRIES} tentativas.")
    return None

# ============================================
# EQUIPAMENTOS (ativos)
# ============================================

def coletar_equipamentos(token: str, session: Optional[requests.Session] = None) -> Optional[List[Dict[str, Any]]]:
    params = {"ativo": "true"}
    dados = chamar_api("GET", "equipamento/list", token, params=params, session=session)
    if not dados:
        return None
    if isinstance(dados, list):
        return dados
    if isinstance(dados, dict) and "items" in dados:
        return dados["items"]
    return [dados]

# ============================================
# RESULTADOS (sampleResult/search) com CHUNK + paginação
# ============================================

def gerar_janelas_data(periodo_dias: int, chunk_dias: int) -> List[Tuple[str, str]]:
    hoje = datetime.now().date()
    inicio = hoje - timedelta(days=periodo_dias)
    janelas = []
    cursor = inicio
    while cursor <= hoje:
        fim_chunk = min(cursor + timedelta(days=chunk_dias - 1), hoje)
        janelas.append((cursor.strftime("%Y-%m-%d"), fim_chunk.strftime("%Y-%m-%d")))
        cursor = fim_chunk + timedelta(days=1)
    return janelas

def coletar_resultados(token: str, periodo_dias: int = PERIODO_DIAS,
                       chunk_dias: int = CHUNK_DIAS,
                       session: Optional[requests.Session] = None) -> List[Dict[str, Any]]:
    todos: List[Dict[str, Any]] = []
    janelas = gerar_janelas_data(periodo_dias, chunk_dias)
    print(f"[Resultados] Janelas geradas: {len(janelas)} (de {janelas[0][0]} até {janelas[-1][1]})")

    for (data_inicio, data_fim) in janelas:
        page = 1
        print(f"[Resultados] Janela {data_inicio} → {data_fim}")
        while True:
            body = {
                "readingStatus": None,
                "markRead": None,
                "startDate": data_inicio,
                "endDate": data_fim,
                "page": page,
                "pageSize": PAGE_SIZE,
            }
            dados = chamar_api("POST", "sampleResult/search", token, body=body, session=session)
            if not dados:
                print(f"[Resultados] Sem dados ou falha na janela {data_inicio} → {data_fim}, página {page}.")
                break

            if isinstance(dados, dict) and isinstance(dados.get("items"), list):
                itens = dados["items"]
            elif isinstance(dados, list):
                itens = dados
            else:
                itens = [dados]

            qtd = len(itens)
            todos.extend(itens)
            print(f"[Resultados] {data_inicio} → {data_fim} | pág {page} | itens {qtd} | acumulado {len(todos)}")

            if qtd < PAGE_SIZE:
                break
            page += 1

    return todos

# ============================================
# ETL: FLATTEN (1 linha = 1 teste por amostra)
# ============================================

def flatten_resultados(resultados: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    linhas: List[Dict[str, Any]] = []

    for s in resultados or []:
        # Metadados da amostra
        sample_id         = s.get("id", "")
        sample_number     = s.get("sampleNumber", "")
        external_code     = s.get("externalCode", "")
        result_date       = s.get("resultDate", "")
        reading_status    = s.get("readingStatus", "")

        vr_status         = get(s, "validResult.resultStatus")
        vr_eval           = fix_mojibake(get(s, "validResult.evaluation"))
        vr_action         = fix_mojibake(get(s, "validResult.inspectionAction"))

        # Equip/cliente/site/compartimento
        equip_model       = get(s, "equipment.model")
        equip_serial      = get(s, "equipment.serial")
        equip_tag         = get(s, "equipment.tag")
        equip_family      = fix_mojibake(get(s, "equipment.family.name"))
        equip_maker       = get(s, "equipment.maker.name")

        site_name         = fix_mojibake(get(s, "site.name") or get(s, "equipment.site.name"))
        site_ext_code     = get(s, "site.externalCode") or get(s, "equipment.site.externalCode")

        customer_name     = fix_mojibake(get(s, "customer.name"))
        customer_id       = get(s, "customer.id")
        operation_name    = get(s, "operation.name")
        lab_name          = fix_mojibake(get(s, "laboratory.name"))
        payment_terms     = s.get("paymentTerms", "")

        comp_name         = fix_mojibake(get(s, "compartment.name") or get(s, "collectionData.compartmentName"))
        comp_type         = fix_mojibake(get(s, "compartment.type.name") or get(s, "collectionData.compartmentType.name"))
        comp_volume       = get(s, "compartment.volume")

        coll_date_sampled = get(s, "collectionData.dateSampled")
        coll_reg_date     = get(s, "collectionData.registrationDate")
        coll_time_type    = get(s, "collectionData.timeType")
        coll_fluid_time   = get(s, "collectionData.fluidTime")
        coll_equipt_time  = get(s, "collectionData.equipmentTime")
        oil_viscosity     = get(s, "collectionData.oil.viscosity.name")
        oil_manufacturer  = get(s, "collectionData.oil.manufacturer.name")
        oil_changed       = get(s, "collectionData.oilChanged")

        test_packages     = fix_mojibake(join_names(s.get("testPackages") or [], "name"))

        tests = s.get("testResults") or []
        if not tests:
            # Linha de rastreabilidade (sem testes)
            linhas.append({
                "sampleId": sample_id, "sampleNumber": sample_number, "externalCode": external_code,
                "resultDate": result_date, "readingStatus": reading_status,
                "validResultStatus": vr_status, "validResultEvaluation": vr_eval, "validResultInspectionAction": vr_action,
                "equipmentModel": equip_model, "equipmentSerial": equip_serial, "equipmentTag": equip_tag,
                "equipmentFamily": equip_family, "equipmentMaker": equip_maker,
                "siteName": site_name, "siteExternalCode": site_ext_code,
                "customerName": customer_name, "customerId": customer_id,
                "operationName": operation_name, "laboratoryName": lab_name,
                "paymentTerms": payment_terms,
                "compartmentName": comp_name, "compartmentType": comp_type, "compartmentVolume": comp_volume,
                "collectionDateSampled": coll_date_sampled, "collectionRegistrationDate": coll_reg_date,
                "collectionTimeType": coll_time_type, "collectionFluidTime": coll_fluid_time,
                "collectionEquipmentTime": coll_equipt_time, "oilViscosity": oil_viscosity,
                "oilManufacturer": oil_manufacturer, "oilChanged": oil_changed,
                "testGroup": "", "testName": "", "testAbbreviation": "", "testUnitOfMeasure": "",
                "testMethod": "", "testValueType": "", "testOrder": "",
                "resultValue_raw": "", "resultValue_num": None, "resultValue_is_numeric": False,
                "resultStatus": "", "testPackages": test_packages
            })
            continue

        for tr in tests:
            test_group       = fix_mojibake(get(tr, "test.testGroup.name"))
            test_name        = fix_mojibake(get(tr, "test.translation.name"))
            test_abbrev      = fix_mojibake(get(tr, "test.translation.abbreviation"))
            test_uom         = fix_mojibake(get(tr, "test.translation.unitOfMeasure"))
            test_method      = fix_mojibake(get(tr, "test.translation.method"))
            test_value_type  = get(tr, "test.valueType")
            test_order       = get(tr, "test.order")
            result_status    = tr.get("resultStatus", "")
            result_value     = tr.get("resultValue", "")

            num, raw, is_num = normalize_decimal(result_value)

            linhas.append({
                "sampleId": sample_id, "sampleNumber": sample_number, "externalCode": external_code,
                "resultDate": result_date, "readingStatus": reading_status,
                "validResultStatus": vr_status, "validResultEvaluation": vr_eval, "validResultInspectionAction": vr_action,

                "equipmentModel": equip_model, "equipmentSerial": equip_serial, "equipmentTag": equip_tag,
                "equipmentFamily": equip_family, "equipmentMaker": equip_maker,
                "siteName": site_name, "siteExternalCode": site_ext_code,
                "customerName": customer_name, "customerId": customer_id,
                "operationName": operation_name, "laboratoryName": lab_name,
                "paymentTerms": payment_terms,

                "compartmentName": comp_name, "compartmentType": comp_type, "compartmentVolume": comp_volume,
                "collectionDateSampled": coll_date_sampled, "collectionRegistrationDate": coll_reg_date,
                "collectionTimeType": coll_time_type, "collectionFluidTime": coll_fluid_time,
                "collectionEquipmentTime": coll_equipt_time, "oilViscosity": oil_viscosity,
                "oilManufacturer": oil_manufacturer, "oilChanged": oil_changed,

                "testGroup": test_group, "testName": test_name, "testAbbreviation": test_abbrev,
                "testUnitOfMeasure": test_uom, "testMethod": test_method,
                "testValueType": test_value_type, "testOrder": test_order,
                "resultValue_raw": raw, "resultValue_num": num, "resultValue_is_numeric": is_num,
                "resultStatus": result_status,
                "testPackages": test_packages,
            })

    return linhas

# ============================================
# SALVAR CSV
# ============================================

def salvar_csv_table(df: pd.DataFrame, nome: str, col_order: Optional[List[str]] = COL_ORDER) -> None:
    ensure_dir(PASTA_SAIDA)
    caminho = os.path.join(PASTA_SAIDA, nome)
    df2 = aplicar_ordem_colunas(df, col_order).fillna("")
    df2.to_csv(
        caminho,
        index=False,
        encoding=CSV_ENCODING,
        sep=CSV_SEPARADOR,
        quoting=QUOTE_MINIMAL  # MINIMAL já cita campos com vírgula
    )
    print(f"[CSV] Salvo: {caminho}")

# ============================================
# VALIDAÇÃO / PREVIEW
# ============================================

def validar_flat(linhas: List[Dict[str, Any]]) -> None:
    if not linhas:
        print("[Validacao] Flatten sem linhas.")
        return
    df = pd.DataFrame(linhas)
    df_preview = aplicar_ordem_colunas(df, COL_ORDER)
    print("\n[Validacao] Preview (5 linhas):")
    print(df_preview.head(5).to_string(index=False))
    print(f"\n[Validacao] Total de linhas (testes): {len(df)}")
    if "sampleId" in df.columns:
        print(f"[Validacao] Amostras distintas: {df['sampleId'].nunique()}")
    if "testName" in df.columns:
        exemplos = sorted(df["testName"].dropna().unique())[:10]
        print(f"[Validacao] Exemplos de testes: {exemplos}")

# ============================================
# PROCESSAR CSV EXISTENTE (coluna 'results')
# ============================================

def parse_python_like_json(text: str) -> Any:
    """
    A coluna 'results' costuma vir como string Python-like (aspas simples, None/True/False).
    Preferimos literal_eval; fallback para JSON.
    """
    txt = str(text).strip()
    # Remove aspas externas se houver
    if (txt.startswith('"') and txt.endswith('"')) or (txt.startswith("'") and txt.endswith("'")):
        txt = txt[1:-1]
    # Primeiro tenta literal_eval (aceita aspas simples e None/True/False)
    try:
        return ast.literal_eval(txt)
    except Exception:
        # Fallback: tokens Python -> JSON
        txt2 = txt.replace("None", "null").replace("True", "true").replace("False", "false")
        return json.loads(txt2)

def processar_csv_results_file(src_file: str) -> List[Dict[str, Any]]:
    """
    Lê CSV de entrada detectando automaticamente o delimitador (',' ou ';')
    e extrai a coluna 'results' com o array serializado.
    """
    sep_in = detectar_sep_csv(src_file)
    print(f"[CSV] Lendo: {src_file} | delimitador detectado: '{sep_in}'")
    df_raw = pd.read_csv(src_file, sep=sep_in, encoding=CSV_ENCODING, engine="c", dtype=str)
    if "results" not in df_raw.columns:
        # Tenta localizar coluna que contenha o array (fallback)
        possiveis = [c for c in df_raw.columns if "results" in c.lower()]
        if possiveis:
            df_raw.rename(columns={possiveis[0]: "results"}, inplace=True)
        else:
            raise RuntimeError("CSV não possui coluna 'results'.")

    all_items: List[Dict[str, Any]] = []
    for s in df_raw["results"].dropna().astype(str).tolist():
        payload = parse_python_like_json(s)
        if isinstance(payload, list):
            all_items.extend(payload)
        elif isinstance(payload, dict) and "items" in payload:
            all_items.extend(payload["items"])
        else:
            # se for um único objeto, adiciona
            if isinstance(payload, dict):
                all_items.append(payload)
    print(f"[CSV] Items extraídos de 'results': {len(all_items)}")
    return all_items

# ============================================
# EXECUÇÃO PRINCIPAL (API)
# ============================================

def executar_via_api():
    print("Iniciando via API:", datetime.now())
    session = requests.Session()
    token = autenticar(session=session)

    # Equipamentos (ativos)
    equipamentos = coletar_equipamentos(token, session=session)
    if equipamentos:
        salvar_csv_table(pd.DataFrame(equipamentos), f"equipamentos_{datetime.now().date()}.csv")
    else:
        print("[Equipamentos] Nenhum dado retornado.")

    # Resultados em janelas + paginação
    resultados = coletar_resultados(token, periodo_dias=PERIODO_DIAS, chunk_dias=CHUNK_DIAS, session=session)
    if resultados:
        # Flatten → CSV
        linhas_flat = flatten_resultados(resultados)
        validar_flat(linhas_flat)
        df_flat = pd.DataFrame(linhas_flat)
        salvar_csv_table(df_flat, f"resultados_flat_{datetime.now().date()}.csv")
    else:
        print("[Resultados] Nenhum dado retornado.")

    print("Finalizado:", datetime.now())

# ============================================
# EXECUÇÃO PRINCIPAL (CSV existente com 'results')
# ============================================

def executar_via_csv(src_file: str):
    print("Iniciando via CSV:", datetime.now())
    resultados = processar_csv_results_file(src_file)
    if not resultados:
        print("[CSV] Nenhum item encontrado em 'results'.")
        return

    # Flatten → CSV
    linhas_flat = flatten_resultados(resultados)
    validar_flat(linhas_flat)
    df_flat = pd.DataFrame(linhas_flat)
    salvar_csv_table(df_flat, f"resultados_flat_{datetime.now().date()}.csv")

    print("Finalizado:", datetime.now())

# ============================================
# MAIN
# ============================================

if __name__ == "__main__":
    ensure_dir(PASTA_SAIDA)
    try:
        if MODE == "csv":
            if not SRC_FILE:
                raise RuntimeError("Defina SRC_FILE=/caminho/arquivo.csv quando MODE=csv.")
            executar_via_csv(SRC_FILE)
        else:
            executar_via_api()
    except Exception as e:
        print("Falha na execução:", e)


Iniciando via API: 2026-01-13 23:13:52.745979
[LOGIN] Status: 400 Bad Request | Content-Type: 
[LOGIN] Body (até 1000 chars): 
[LOGIN] Headers: {'Date': 'Tue, 13 Jan 2026 23:13:52 GMT'}
[LOGIN] Autenticado com payload: ['username', 'password']
[API] GET equipamento/list tentativa 1/3 Status: 405 Method Not Allowed | Content-Type: application/xml;charset=UTF-8
[API] GET equipamento/list tentativa 1/3 Body (até 1000 chars): <Map><timestamp>1768346033091</timestamp><status>405</status><error>Method Not Allowed</error><path>/api/v1/equipamento/list</path></Map>
[API] GET equipamento/list tentativa 1/3 Headers: {'Date': 'Tue, 13 Jan 2026 23:13:52 GMT'}
[Equipamentos] Nenhum dado retornado.
[Resultados] Janelas geradas: 12 (de 2025-01-13 até 2026-01-13)
[Resultados] Janela 2025-01-13 → 2025-02-12
[Resultados] 2025-01-13 → 2025-02-12 | pág 1 | itens 1 | acumulado 1
[Resultados] Janela 2025-02-13 → 2025-03-15
[Resultados] 2025-02-13 → 2025-03-15 | pág 1 | itens 1 | acumulado 2
[Resultados] Jan